DATA MINING SUPERMARKET_SALES

1. Liệu giới tính có liên quan đến loại sản phẩm được mua ? Dự đoán loại sản phẩm mà người phụ nữ tiếp theo sẽ mua.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
print('20127250 \nHo Thi Thanh Ngan')